<a href="https://colab.research.google.com/github/Jack3690/pydol/blob/main/notebooks/pydol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pipeline**

In [ ]:
from jwst.pipeline import Detector1Pipeline, Image2Pipeline, Image3Pipeline
import jwst.associations

from glob import glob

import os
from crds import client
import jwst
import multiprocessing as mp
from pathlib import Path

crds_dir = Path(__file__).parent.joinpath('CRDS')/'crds_cache'
os.makedirs(crds_dir, exist_ok=True)
os.environ['CRDS_PATH'] = str(crds_dir)
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"
client.set_crds_server("https://jwst-crds.stsci.edu")


In [3]:
class jpipe():
    def __init__(self, input_files=[], out_dir='.',
                 crds_context="jwst_1241.pmap"):
        """
            Parameters
            ----------
            input_files: list,
                         Input list of level 0 '_uncal.fits' files. 
                         Recommended: /data/stage0/
            out_dir: str,
                     Output directory. 
                     Recommended: The directory that contains /data/stage0/
                     Pipeline will create /data/stage1/ and /data/stage2/
                     
            crds_context: str,
                          Reference context for JWST pipeline from CRDS.

              Returns
              -------
                  None
                        
        """
        if len(input_files)<1:
            raise Exception("Input files list CANNOT be empty!")
        self.input_files = input_files
        self.out_dir = out_dir
        os.makedirs(out_dir + '/data/stage1/', exist_ok=True)
        os.makedirs(out_dir + '/data/stage2/', exist_ok=True)
        os.environ["CRDS_CONTEXT"] = crds_context

    def stage1(self, filename):
        """
            Parameters
            ----------
            filename: str,
                      path to the level 0 "_uncal.fits" file
            Returns
            -------
                None
        """
        # Instantiate the pipeline
        img1 = Detector1Pipeline()
        # Specify where the output should go
        img1.output_dir = self.out_dir + '/data/stage1/'
        # Save the final resulting _rate.fits files
        img1.save_results = True
        #No of cores
        img1.jump.maximum_cores = f'{mp.cpu_count()-1}'
        # Run the pipeline on an input list of files
        img1(filename)
        
    def stage2(self, filename):
        """
            Parameters
            ----------
            filename: str,
                      path to the level 1 "_rate.fits" file
            Returns
            -------
                None
        """
        # Instantiate the pipeline
        img2 = Image2Pipeline()
        # Specify where the output should go
        img2.output_dir = self.out_dir + '/data/stage2/'
        # Save the final resulting _rate.fits files
        img2.save_results = True
        # Run the pipeline on an input list of files
        img2(filename)

    def __call__(self):
        """
            Runs the JWST Stage 1 and Stage 2 pipeline for generating
            '_cal.fits' files
        """
        uncal_files = [i for i in self.input_files if 'uncal' in i ]
        for f in uncal_files:
            o = f.replace('stage0','stage1')
            o = o.replace('uncal','rate')
            if not os.path.exists(o):
                self.stage1(f)

        rate_files = glob(self.out_dir + '/data/stage1/*_rate.fits')
        rate_files_ = []
        for f in rate_files:
            o = f.replace('stage1','stage2')
            o = o.replace('rate','cal')
            if not os.path.exists(o):
                rate_files_.append(f)
            
        if len(rate_files_)>0:
            with mp.Pool(mp.cpu_count()-1) as p:
                p.map(self.stage2, rate_files_)




# **Photometry**

In [ ]:
import os
from glob import glob
from astropy.table import Table
from astropy.wcs import WCS
from astropy.io import fits
import numpy as np
import multiprocessing as mp
from pathlib import Path
import subprocess

param_dir = str(Path(__file__).parent.joinpath('params'))
script_dir = str(Path(__file__).parent.joinpath('scripts'))

In [ ]:

def nircam_phot(cal_files, name='f200w',output_dir='.', drz_path='.', ):
    subprocess.run(["nircammask", f"{drz_path}.fits"], shell=True) 
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)

    # Generating directories
    exps = []
    for i,f in enumerate(cal_files):
        out_dir = f.split('/')[-1].split('.')[0]

        if not os.path.exists(f'{output_dir}/{out_dir}'):
            os.mkdir(f'{output_dir}/{out_dir}')
        if not os.path.exists(f"{output_dir}/{out_dir}/data.fits"):
            subprocess.run(["cp", f"{f}", f"{output_dir}/{out_dir}/data.fits"], shell=True)

        exps.append(f'{output_dir}/{out_dir}')

    # Applying NIRCAM Mask
    for f in exps:
        if not os.path.exists(f"{f}/data.sky.fits"):
            subprocess.run(["nircammask", f"{f}/data.fits"], shell=True)
            subprocess.run(["calcsky", f"{f}/data", "10", "25", "2", "2.25", "2.00"], shell=True)

    # Preparing Parameter file DOLPHOT NIRCAM
    with open(f"{param_dir}/nircam_dolphot.param") as f:
                dat = f.readlines()

    dat[0] = f'Nimg = {len(exps)}                #number of images (int)\n'
    dat[4] = f'img0_file = {drz_path}\n'

    for i,f in enumerate(exps):
        dat[5+i] = f'img{i+1}_file = {f}/data           #image {i+1}\n'

    out_id = np.random.random()
    with open(f"{param_dir}/nircam_dolphot_{out_id}.param", 'w', encoding='utf-8') as f:
        f.writelines(dat)
        
    if not os.path.exists(f"{output_dir}/{name}_photometry.fits"):
        # Running DOLPHOT NIRCAM
         subprocess.run(["dolphot", f"{output_dir}/out", f"-p{param_dir}/nircam_dolphot_{out_id}.param"], shell=True)

    # Generating Astropy FITS Table
   
    subprocess.run(["python", f"{script_dir}to_table.py", "--o", f"{name}_photometry", "--n", f"{len(exps)}", "--f", f"{output_dir}/out"])

    phot_table = Table.read(f"{output_dir}/{name}_photometry.fits")
    phot_table.rename_columns(['mag_vega'],[f'mag_vega_F200W'])

    # Assingning RA-Dec using reference image
    hdu = fits.open(f"{drz_path}.fits")[1]

    wcs = WCS(hdu.header)
    positions = np.transpose([phot_table['x'] - 0.5, phot_table['y']-0.5])

    coords = np.array(wcs.pixel_to_world_values(positions))

    phot_table['ra']  = coords[:,0]
    phot_table['dec'] = coords[:,1]

    # Filtering stellar photometry catalog using Warfield et.al (2023)
    phot_table1 = phot_table[ (phot_table['sharpness']**2   <= 0.01) &
                                (phot_table['obj_crowd']    <=  0.5) &
                                (phot_table['flags']        <=    2) &
                                (phot_table['type']         <=    2)]

    phot_table2 = phot_table[ ~((phot_table['sharpness']**2 <= 0.01) &
                                (phot_table['obj_crowd']    <=  0.5) &
                                (phot_table['flags']        <=    2) &
                                (phot_table['type']         <=    2))]
    print('NIRCAM SHORT')
    phot_table.write(f'{output_dir}/{name}_photometry.fits', overwrite=True)
    phot_table1.write(f'{output_dir}/{name}_photometry_filt.fits', overwrite=True)
    phot_table2.write(f'{output_dir}/{name}_photometry_rej.fits', overwrite=True)
    


